# Machine Learning Experiment \#1

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

from operator import add

import bisect
import datetime
import math
import random
import time

from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout

In [ ]:
datafile = r'''..\Data\IVE_bidaskvol1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh', 'AskLow', 'AskClose', 'Volume']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [ ]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,1,2,0,0,0)].copy()
#pricedata = pricedata[pricedata['DateTime'] < datetime.datetime(2019,1,3,0,0,0)].copy()
pricedata = pricedata.reset_index()
del pricedata['index']

In [ ]:
# Method that returns a tuple containing numpy arrays of NN inputs and rewards from the specified date range
def get_inputs_and_rewards(start_date, end_date, pricedata):
    inputs = []
    rewards = []
    days = []
    
    # Get list of days
    curr_day = start_date
    while curr_day < end_date:
        days.append(curr_day)
        curr_day = curr_day + datetime.timedelta(days=1)

    # Iterate over days and get inputs/rewards
    for day in days:
        daypricedata = pricedata[pricedata['DateTime'] > pd.Timestamp(day)].copy()
        daypricedata = daypricedata[daypricedata['DateTime'] < pd.Timestamp(day) + datetime.timedelta(days=1)].copy()
        daypricedata = daypricedata.reset_index()
        N = agent.inputs + 1
        n = 10
        M = len(daypricedata)-N-n

        for index in range(0,M):
            diffs = np.diff(daypricedata['BidClose'][index:index+N])
            #vols = np.array(daypricedata['Volume'][index:index+N-1])
            #inputs.append(np.concatenate((diffs, vols)))
            inputs.append(diffs)
            rewards.append(daypricedata['BidClose'][index+N+n] - daypricedata['BidClose'][index+N])
            
    return (np.array(inputs), np.array(rewards))

In [ ]:
class DQNAgent(object):

    def __init__(self):
        self.inputs = 20
        self.outputs = 1
        self.model = self.network()

    def network(self, weights=None):
        model = Sequential()
        
        num_layers = 4
        neurons_per_layer = 20
        dropout = 0.0
        reg = 0.00
        act = 'tanh'
        output_act = 'linear'
        learning = 0.0025
        opt = optimizers.Adam(learning)
        
        if num_layers == 0:
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=self.outputs, activation=output_act, input_dim=self.inputs,
                        kernel_regularizer=regularizers.l1(reg),
                        activity_regularizer=regularizers.l1(reg)))
        else:
            model.add(Dense(units=neurons_per_layer, activation=act, input_dim=self.inputs,
                            kernel_regularizer=regularizers.l1(reg),
                            activity_regularizer=regularizers.l1(reg)))
            
            for ii in range(num_layers - 1):
                if dropout > 0:
                    model.add(Dropout(dropout))
                model.add(Dense(units=neurons_per_layer, activation=act,
                                kernel_regularizer=regularizers.l1(reg),
                                activity_regularizer=regularizers.l1(reg)))
                
            if dropout > 0:
                    model.add(Dropout(dropout))
            model.add(Dense(units=self.outputs, activation=output_act,
                            kernel_regularizer=regularizers.l1(reg),
                            activity_regularizer=regularizers.l1(reg))) 
    
        model.compile(loss='mse', optimizer=opt)

        return model    
        
    # Train the NN    
    def train(self, inputs, outputs, epochs):
        return self.model.fit(inputs, outputs, epochs=epochs, verbose=0, batch_size=100)
    
    # Train and evaluate at intervals on test data to see if the losses on test data are decreasing
    def train_and_evaluate(self, training_inputs, training_rewards, testing_inputs, testing_rewards, epochs, epoch_eval_interval):
        intervals = int(np.ceil(epochs/epoch_eval_interval))
        testing_losses = []
        testing_losses_idx = []
        training_losses = []
        
        # Get the performance on the testing data before training
        testing_losses.append(agent.model.evaluate(testing_inputs, testing_rewards))
        testing_losses_idx.append(0)
        
        # Train in intervals of epochs so we can evaluate the NN performance on testing data in between
        for ii in range(intervals):
            history = agent.train(training_inputs, training_rewards, epochs=epoch_eval_interval)
            interval_training_losses = history.history['loss']
            training_losses = training_losses + interval_training_losses
            interval_testing_loss = agent.model.evaluate(testing_inputs, testing_rewards, verbose=0)
            testing_losses.append(interval_testing_loss)
            testing_losses_idx.append((ii+1)*epoch_eval_interval)
            print('At epoch ', (ii+1)*epoch_eval_interval, ' out of ', epochs)
            print('   Mean training loss: ', round(np.mean(interval_training_losses), 5))
            print('   Testing loss:       ', round(np.mean(interval_testing_loss), 5))
        
        # Convert to np.array
        testing_losses = np.array(testing_losses)
        testing_losses_idx = np.array(testing_losses_idx)
        
        return (training_losses, testing_losses, testing_losses_idx)
            

In [ ]:
agent = DQNAgent()

# Indexs for training and testing ranges
training_start_idx = 0
training_end_idx = np.floor(len(pricedata)/2)
testing_start_idx = training_end_idx + 1
testing_end_idx = len(pricedata) -  1

# Training date range
training_start_date = pricedata['DateTime'][training_start_idx].date()
training_end_date = pricedata['DateTime'][training_end_idx].date()

# Testing date range
testing_start_date = pricedata['DateTime'][testing_start_idx].date()
testing_end_date = pricedata['DateTime'][testing_end_idx].date()

# Training inputs and rewards
(training_inputs, training_rewards) = get_inputs_and_rewards(start_date=training_start_date,
                                                             end_date=training_end_date,
                                                             pricedata=pricedata)
# Testing inputs and rewards
(testing_inputs, testing_rewards) = get_inputs_and_rewards(start_date=testing_start_date,
                                                           end_date=testing_end_date,
                                                           pricedata=pricedata)
# Amount of training time
epochs = 200
epoch_eval_interval = 20

# Train the NN
# history = agent.train(training_inputs, training_rewards, epochs=1000)
# training_losses = history.history['loss']
(training_losses, testing_losses, testing_losses_idx) = agent.train_and_evaluate(training_inputs=training_inputs,
                                                                                 training_rewards=training_rewards,
                                                                                 testing_inputs=testing_inputs,
                                                                                 testing_rewards=testing_rewards,
                                                                                 epochs=epochs,
                                                                                 epoch_eval_interval=epoch_eval_interval)

# Get the performance on the training and testing data after training
training_predictions = agent.model.predict_on_batch(training_inputs)
testing_predictions = agent.model.predict_on_batch(testing_inputs)

In [ ]:
training_idx_plot = [ii for ii in range(len(training_predictions))]
testing_idx_plot = [ii for ii in range(len(testing_predictions))]

training_predictions_plot = [w[0] for w in training_predictions]
testing_predictions_plot = [w[0] for w in testing_predictions]

print('Mean Training Prediction: ', np.mean(training_predictions_plot))
print('Mean Training Reward:     ', np.mean(training_rewards))
print('Mean Testing Prediction:  ', np.mean(testing_predictions_plot))
print('Mean Testing Reward:      ', np.mean(testing_rewards))

In [ ]:
class PredictionsChart:
    def __init__(self, idx, rewards, predictions, title='Plot'):
        self.data = [
                dict(
                    type = 'scatter',
                    x = idx,
                    y = rewards,
                    mode='lines',
                    name='Rewards'
                ),
                
                dict(
                    type = 'scatter',
                    x = idx,
                    y = predictions,
                    mode='lines',
                    name='Predictions'
                ),
               ]
        
        self.layout = go.Layout(
                        title=title,
                        
                        xaxis=dict(
                            title='Epochs',
                        ),

                        yaxis=dict(
                            title='Reward/Prediction',
                        ),
                    )
        
        self.chart = go.FigureWidget(data=self.data, layout=self.layout)
        
        self.display = self.chart
    
training_predictions_chart = PredictionsChart(training_idx_plot, training_rewards, training_predictions_plot, title='Training')
training_predictions_chart.display

In [ ]:
testing_predictions_chart = PredictionsChart(testing_idx_plot, testing_rewards, testing_predictions_plot, title='Testing')
testing_predictions_chart.display

In [ ]:
class LossesChart:
    def __init__(self):
        data = [
                dict(
                    type = 'scatter',
                    x = [ii for ii in range(len(training_losses))],
                    y = training_losses,
                    mode='lines',
                    name='Training Data Losses'
                ),
            
                dict(
                    type = 'scatter',
                    x = testing_losses_idx,
                    y = testing_losses,
                    mode='lines',
                    name='Testing Data Losses'
                ),
               ]
        
        layout = go.Layout(
                        title = 'Losses',

                        xaxis=dict(
                            title='Epochs',
                        ),

                        yaxis=dict(
                            title='Loss',
                        ),
                    )
        
        self.chart = go.FigureWidget( data=data, layout=layout )
        
        self.display = self.chart
        
losses_chart = LossesChart()
losses_chart.display

In [ ]:
class NormalizedLossesChart:
    def __init__(self):
        data = [
                dict(
                    type = 'scatter',
                    x = [ii for ii in range(len(training_losses))],
                    y = training_losses/training_losses[0],
                    mode='lines',
                    name='Training Data Losses'
                ),
                dict(
                    type = 'scatter',
                    x = testing_losses_idx,
                    y = testing_losses/testing_losses[0],
                    mode='lines',
                    name='Testing Data Losses'
                ),
               ]
        
        
        layout = go.Layout(
                        title = 'Normalized Losses',

                        xaxis=dict(
                            title='Epochs',
                        ),

                        yaxis=dict(
                            title='Normalized Loss',
                        ),
                    )
        
        self.chart = go.FigureWidget( data=data, layout=layout )
        
        self.display = self.chart
        
normalized_losses_chart = NormalizedLossesChart()
normalized_losses_chart.display

# To-Do List

* Extract volume per minute data from tick data
* LossesChart/etc. applied to separate test data
* Try feeding a vector of indicators instead of raw price data
* Implement actual buy/sell strategy based on NN output, evaluate performance